# Search Algorithms

**Manual Search:** Best for small, simple models or when expert knowledge is available; less computational overhead but may miss optimal configurations.

**Grid Search:** Comprehensive and systematic but computationally expensive; suitable for smaller hyperparameter spaces.

**Random Search:** Efficient for high-dimensional spaces; balances exploration and computational efficiency, especially when the optimal configuration is not well-known.

## Grid Search

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import (
    GridSearchCV,
    train_test_split,
)

In [ ]:
# Step 1: Set up the model
# Initializes a Gradient Boosting Classifier with a fixed random state for reproducibility
gbm = GradientBoostingClassifier(random_state=0)

# Step 2: Determine the hyperparameter space
# Defines the hyperparameter grid with options for number of estimators, min samples split, and max depth
param_grid = dict(
    n_estimators=[10, 20, 50, 100],          # Number of boosting stages
    min_samples_split=[0.1, 0.3, 0.5],       # Minimum fraction of samples required to split a node
    max_depth=[1, 2, 3, 4, None],            # Maximum depth of the individual trees
)

# Step 3: Calculate and print the total number of hyperparameter combinations
# Calculates and prints the total number of hyperparameter combinations to be evaluated
print('Number of hyperparam combinations: ',
      len(param_grid['n_estimators']) * len(param_grid['min_samples_split']) * len(param_grid['max_depth']))

# Step 4: Set up the search
# Configures Grid Search with the model, hyperparameters, ROC AUC scoring, and 5-fold cross-validation
search = GridSearchCV(gbm, param_grid, scoring='roc_auc', cv=5, refit=True)

# Step 5: Find best hyperparameters
# Fits the model to the training data to find the best hyperparameters
search.fit(X_train, y_train)

# Step 6: Retrieve the best hyperparameters
# Retrieves the best hyperparameters identified by the Grid Search
search.best_params_

# Step 7: Convert the search results into a DataFrame
# Converts the full results of the Grid Search into a DataFrame
results = pd.DataFrame(search.cv_results_)

# Step 8: Print the shape of the results DataFrame
# Prints the shape of the DataFrame containing all model evaluations
print(results.shape)

# Step 9: Display the first few rows of the results DataFrame
# Displays the first few rows of the DataFrame
results.head()

# Step 10: Sort the models based on performance
# Sorts the DataFrame by the mean test score in descending order to find the best performing models
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

# Step 11: Reset the index of the DataFrame after sorting
# Resets the index of the DataFrame after sorting
results.reset_index(drop=True, inplace=True)

# Step 12: Display the top performing models
# Displays the top performing models with their parameters and scores
results[[
    'param_max_depth', 'param_min_samples_split', 'param_n_estimators',
    'mean_test_score', 'std_test_score',
]].head()

# Step 13: Display the bottom performing models
# Displays the bottom performing models with their parameters and scores
results[[
    'param_max_depth', 'param_min_samples_split', 'param_n_estimators',
    'mean_test_score', 'std_test_score',
]].tail()

In [ ]:
# let's make a function to evaluate the model performance based on
# single hyperparameters
# Defines a function to summarize the model's performance (mean and std) for a specific hyperparameter
def summarize_by_param(hparam):

    # Groups the results by the specified hyperparameter and calculates the mean and standard deviation of test scores
    tmp = pd.concat([
        results.groupby(hparam)['mean_test_score'].mean(),
        results.groupby(hparam)['mean_test_score'].std(),
    ], axis=1)

    # Renames the columns for clarity
    tmp.columns = ['mean_test_score', 'std_test_score']

    return tmp  # Returns the summary DataFrame

# performance change for n_estimators
# Evaluates the performance for different values of 'n_estimators'
tmp = summarize_by_param('param_n_estimators')

tmp.head()  # Displays the first few rows of the summarized results

# Plots the mean test score for 'n_estimators' with error bars representing standard deviation
tmp['mean_test_score'].plot(yerr=[tmp['std_test_score'], tmp['std_test_score']], subplots=True)
plt.ylabel('roc-auc')  # Labels the y-axis to show ROC AUC

# performance change for max_depth
# Evaluates the performance for different values of 'max_depth'
tmp = summarize_by_param('param_max_depth')

# Plots the mean test score for 'max_depth' with error bars representing standard deviation
tmp['mean_test_score'].plot(yerr=[tmp['std_test_score'], tmp['std_test_score']], subplots=True)
plt.ylabel('roc-auc')  # Labels the y-axis to show ROC AUC

# performance change for min_samples_split
# Evaluates the performance for different values of 'min_samples_split'
tmp = summarize_by_param('param_min_samples_split')

# Plots the mean test score for 'min_samples_split' with error bars representing standard deviation
tmp['mean_test_score'].plot(yerr=[tmp['std_test_score'], tmp['std_test_score']], subplots=True)

Once receive the 1st itertion of your best hyper parameters, then you should use them to find the next batch using the given values or do further experiments

In [ ]:
# determine the hyperparameter space
param_grid = dict(
    n_estimators=[60, 80, 100, 120],
    max_depth=[2,3],
    loss = ['log_loss', 'exponential'],
    )

### Nested hyper parameter spaces

In [ ]:
# set up the model
svm = SVC(random_state=0)

# determine the hyperparameter space
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

## Random Search

**Assumptions**

**Random Search Considerations:**
* Choosing a Computational Budget:
The computational budget (e.g., number of iterations, time spent) is chosen independently (or mostly independently) of the number of hyperparameters and possible values. This gives flexibility in managing resources.

**Efficiency with Non-Influential Parameters:**
* Adding hyperparameters that do not influence the model's performance does not reduce the efficiency of the search, provided that enough iterations are performed. Random Search naturally focuses more on important parameters during the search process.

**Continuous Distributions for Hyperparameters:**
* To take full advantage of Random Search’s random sampling, it is crucial to specify a continuous distribution for the hyperparameters. This ensures that Random Search explores the space more effectively rather than being limited to a small set of predefined values.

### Sklearn

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split,
)

In [ ]:
# Step 1: Set up the model
# Initializes a Gradient Boosting Classifier with a fixed random state for reproducibility
gbm = GradientBoostingClassifier(random_state=0)

# Step 2: Determine the hyperparameter space
# Defines a hyperparameter grid with random distributions for the number of estimators,
# min samples split, max depth, and loss function
param_grid = dict(
    n_estimators=stats.randint(10, 120),           # Random integers for number of boosting stages
    min_samples_split=stats.uniform(0, 1),         # Uniform distribution for minimum fraction of samples to split a node
    max_depth=stats.randint(1, 5),                 # Random integers for maximum depth of trees
    loss=('log_loss', 'exponential'),              # Categorical options for loss function
)

# Step 3: Set up the randomized search
# Configures Randomized Search with the model, hyperparameters, ROC AUC scoring, 5-fold cross-validation,
# and 60 iterations for random search
search = RandomizedSearchCV(gbm,
                            param_grid,
                            scoring='roc_auc',
                            cv=5,                # 5-fold cross-validation
                            n_iter=60,           # Number of random iterations
                            random_state=10,     # Seed for reproducibility
                            n_jobs=4,            # Parallelize across 4 jobs
                            refit=True)          # Refits best model to entire dataset

# Step 4: Find best hyperparameters
# Fits the model to the training data to find the best hyperparameters
search.fit(X_train, y_train)

# Step 5: Retrieve best hyperparameters
# The best hyperparameters are stored in an attribute
search.best_params_

# Step 6: Find the data for all models evaluated
# Converts the full results of the Randomized Search into a DataFrame
results = pd.DataFrame(search.cv_results_)

# Step 7: Print the shape of the DataFrame
# Displays the shape of the results DataFrame
print(results.shape)

# Step 8: Display the first few rows of the DataFrame
# Shows the first few rows of the DataFrame
results.head()


### Scikit-Optimize

**Procedure**

To tune the hyper-parameters of our model we need to:

* define a model
* decide which parameters to optimize
* define the objective function we want to minimize.

**NOTE**

Scikit-Optimize will always minimize the objective function, so if we want to maximize a function, for example the roc-auc, we need to negate the metric. Thus, instead of maximizing the roc-auc, we minimize the -roc-auc.

In [ ]:
from skopt import dummy_minimize # for the randomized search
from skopt.plots import plot_convergence
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

In [ ]:
# Step 1: Determine the hyperparameter space
# Defines a hyperparameter grid for Scikit-Optimize with integer, real, and categorical distributions
param_grid = [
    Integer(10, 120, name="n_estimators"),           # Integer range for the number of boosting stages
    Real(0, 0.999, name="min_samples_split"),        # Real range for the minimum fraction of samples to split a node
    Integer(1, 5, name="max_depth"),                 # Integer range for maximum depth of trees
    Categorical(['log_loss', 'exponential'], name="loss"),  # Categorical options for loss function
]

# Step 2: Check the type of the hyperparameter grid
# Verifies that the parameter grid is a list
type(param_grid)

# Step 3: Set up the Gradient Boosting Classifier
# Initializes a Gradient Boosting Classifier with a fixed random state for reproducibility
gbm = GradientBoostingClassifier(random_state=0)

In [ ]:
# Step 1: Design a function to maximize accuracy using GBM with cross-validation
# The decorator `use_named_args` allows the objective function to receive hyperparameters as keyword arguments
@use_named_args(param_grid)
def objective(**params):

    # Step 2: Update the GBM model with new parameters
    # Set the parameters of the Gradient Boosting Model to the current set of hyperparameters
    gbm.set_params(**params)

    # Step 3: Cross-validation to evaluate the model
    # Perform 3-fold cross-validation on the training data to compute the accuracy for the current parameters
    value = np.mean(
        cross_val_score(
            gbm,
            X_train,        # Training features
            y_train,        # Training labels
            cv=3,           # 3-fold cross-validation
            n_jobs=-4,      # Parallel processing with 4 jobs
            scoring='accuracy')  # Scoring based on accuracy
    )

    # Step 4: Negate the value since the optimizer minimizes the objective
    # Return the negative of the accuracy as the objective function minimizes the value
    return -value

In [ ]:
# Step 1: Perform the randomized search using dummy_minimize
# Runs a minimization search to find the best parameters by calling the objective function multiple times
search = dummy_minimize(
    objective,    # The objective function to minimize
    param_grid,   # The hyperparameter space to explore
    n_calls=50,   # Number of evaluations for the objective function
    random_state=0,  # Ensures reproducibility
)

# Step 2: Display the best score (negative of accuracy)
# Prints the best objective function value found (note: it is the negative of the accuracy)
"Best score=%.4f" % search.fun

# Step 3: Print the best hyperparameters found
# Displays the best hyperparameter values identified during the search
print("""Best parameters:
=========================
- n_estimators=%d
- min_samples_split=%.6f
- max_depth=%d
- loss=%s""" % (search.x[0],  # Best n_estimators
                search.x[1],  # Best min_samples_split
                search.x[2],  # Best max_depth
                search.x[3]))  # Best loss function

# Step 4: Plot the convergence of the search
# Visualizes the convergence of the objective function value over the iterations
plot_convergence(search)

### Randomized Search with Hyperopt


**Procedure**

To tune the hyper-parameters of our model we need to:

* define a model
* decide which parameters to optimize
* define the objective function we want to minimize.

In [ ]:
import xgboost as xgb  # Imports the XGBoost library for gradient boosting models
from hyperopt import hp, rand, fmin, Trials  # Imports necessary modules from Hyperopt for optimization

# hp: define the hyperparameter space
# rand: random search
# fmin: optimization function
# Trials: to evaluate the different searched hyperparameters

# Step 1: Set up the hyperparameter space
# Defines a dictionary specifying the hyperparameter ranges to search
param_grid = {
    'n_estimators': hp.quniform('n_estimators', 200, 2500, 100),  # Number of boosting rounds, ranges from 200 to 2500, in steps of 100
    'max_depth': hp.uniform('max_depth', 1, 10),  # Maximum depth of the trees, continuous values from 1 to 10
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.99),  # Learning rate for model updates, ranges from 0.01 to 0.99
    'booster': hp.choice('booster', ['gbtree', 'dart']),  # Choice of booster: 'gbtree' or 'dart'
    'gamma': hp.quniform('gamma', 0.01, 10, 0.1),  # Minimum loss reduction for further splits, ranges from 0.01 to 10, in steps of 0.1
    'subsample': hp.uniform('subsample', 0.50, 0.90),  # Subsample ratio for the training data, continuous values from 0.50 to 0.90
    'colsample_bytree': hp.uniform('colsample_bytree', 0.50, 0.99),  # Subsample ratio for features (columns) used for each tree, from 0.50 to 0.99
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.50, 0.99),  # Subsample ratio for features used at each tree level, from 0.50 to 0.99
    'colsample_bynode': hp.uniform('colsample_bynode', 0.50, 0.99),  # Subsample ratio for features used at each node split, from 0.50 to 0.99
    'reg_lambda': hp.uniform('reg_lambda', 1, 20)  # L2 regularization term (lambda), from 1 to 20
}

In [ ]:
# Step 1: Define the objective function
# The objective function takes hyperparameters as input to evaluate model performance
def objective(params):

    # Step 2: Create a dictionary for hyperparameters
    # Constructs a dictionary to map hyperparameter values for the XGBoost model
    params_dict = {
        'n_estimators': int(params['n_estimators']),  # Converts n_estimators to int, as XGBoost requires integers
        'max_depth': int(params['max_depth']),        # Converts max_depth to int, as XGBoost requires integers
        'learning_rate': params['learning_rate'],      # Learning rate value from the input parameters
        'booster': params['booster'],                  # Booster type (e.g., 'gbtree' or 'dart') from the input parameters
        'gamma': params['gamma'],                      # Minimum loss reduction for further splits from the input parameters
        'subsample': params['subsample'],              # Subsample ratio of the training data from the input parameters
        'colsample_bytree': params['colsample_bytree'],  # Subsample ratio of features for each tree from the input parameters
        'colsample_bylevel': params['colsample_bylevel'],  # Subsample ratio of features at each tree level from the input parameters
        'colsample_bynode': params['colsample_bynode'],  # Subsample ratio of features at each node from the input parameters
        'random_state': 1000,                           # Sets a random state for reproducibility
    }

    # Step 3: Initialize the XGBoost Classifier
    # Initializes the XGBoost classifier with the specified hyperparameters
    gbm = xgb.XGBClassifier(**params_dict)

    # Step 4: Perform cross-validation
    # Evaluates model performance using cross-validation with accuracy scoring
    score = cross_val_score(gbm, X_train, y_train,
                            scoring='accuracy', cv=5, n_jobs=4).mean()  # Averages the accuracy scores across 5 folds

    # Step 5: Negate the score for minimization
    # Returns the negative of the accuracy score to minimize it during optimization
    return -score

In [ ]:
# Step 1: Perform minimization with hyperopt
# Uses the fmin function to find the best hyperparameters by minimizing the objective function
search = fmin(
    fn=objective,                                  # The objective function to minimize
    space=param_grid,                             # The hyperparameter space defined earlier
    max_evals=50,                                 # The maximum number of evaluations to perform
    rstate=np.random.default_rng(42),            # Sets the random state for reproducibility
    algo=rand.suggest,                            # Specifies the algorithm to use for random search
)

# Step 2: Check the type of the search result
# Retrieves the type of the search result, which should be a dictionary
type(search)

# Step 3: Display the best parameters found by fmin
# Displays the dictionary of the best hyperparameters found
search

# Step 4: Create a dictionary for the best hyperparameters
# Maps the search results to the format required for the XGBoost model
best_hp_dict = {
    'n_estimators': int(search['n_estimators']),  # Converts n_estimators to int, as XGBoost requires integers
    'max_depth': int(search['max_depth']),        # Converts max_depth to int, as XGBoost requires integers
    'learning_rate': search['learning_rate'],      # Learning rate value from the search results
    'booster': 'gbtree',                          # Specifies the booster type
    'gamma': search['gamma'],                      # Minimum loss reduction for further splits from the search results
    'subsample': search['subsample'],              # Subsample ratio of the training data from the search results
    'colsample_bytree': search['colsample_bytree'],  # Subsample ratio of features for each tree from the search results
    'colsample_bylevel': search['colsample_bylevel'],  # Subsample ratio of features at each tree level from the search results
    'colsample_bynode': search['colsample_bynode'],  # Subsample ratio of features at each node from the search results
    'random_state': 1000,                          # Sets a random state for reproducibility
}

# Step 5: Initialize the final XGBoost Classifier
# Creates an XGBoost model using the best hyperparameters obtained from the search
gbm_final = xgb.XGBClassifier(**best_hp_dict)

# Step 6: Train the final model
# Fits the model to the training data using the best hyperparameters
gbm_final.fit(X_train, y_train)

# Step 7: Make predictions on the training set
# Predicts the target values for the training set
X_train_preds = gbm_final.predict(X_train)

# Step 8: Make predictions on the test set
# Predicts the target values for the test set
X_test_preds = gbm_final.predict(X_test)

# Step 9: Calculate and print training accuracy
# Computes and displays the accuracy of the model on the training set
print('Train accuracy: ', accuracy_score(y_train, X_train_preds))

# Step 10: Calculate and print test accuracy
# Computes and displays the accuracy of the model on the test set
print('Test accuracy: ', accuracy_score(y_test, X_test_preds))

### Trial

We can use Trials if we want to look into the search, and the performance values encountered during the process.



In [ ]:
# Step 1: Initialize Trials
# Creates an empty Trials object to store the results of the hyperparameter optimization process
trials = Trials()

# Step 2: Execute Hyperparameter Optimization
# Uses fmin to find the best hyperparameters by minimizing the objective function
second_search = fmin(
    fn=objective,                         # The objective function to minimize
    space=param_grid,                    # The hyperparameter space defined earlier
    max_evals=50,                        # Specifies the maximum number of evaluations to perform
    rstate=np.random.default_rng(42),    # Sets the random state for reproducibility
    algo=rand.suggest,                   # Uses a randomized search algorithm
    trials=trials                        # Associates the trials object to track results
)

# Step 3: View Best Hyperparameters
# Displays the best hyperparameters found during the optimization process
second_search

# Step 4: Retrieve Best Hyperparameters from Trials
# Accesses the index of the best trial from the trials object, representing the optimal hyperparameters
trials.argmin

# Step 5: View Hyperparameter Combinations
# Converts the hyperparameter combinations tested during the search into a pandas DataFrame and displays the first few rows
pd.DataFrame(trials.vals).head()

# Step 6: View Results
# Converts the results of each trial into a DataFrame and displays the first few rows
pd.DataFrame(trials.results).head()

# Step 7: Combine Hyperparameters and Results
# Combines hyperparameter combinations and their corresponding results into a single DataFrame, sorted by loss
results = pd.concat([
    pd.DataFrame(trials.vals),           # DataFrame of hyperparameter values
    pd.DataFrame(trials.results)],       # DataFrame of results
    axis=1,                               # Concatenate along the columns
).sort_values(by='loss', ascending=False).reset_index(drop=True)  # Sort by loss in descending order and reset the index

# Step 8: Display Combined Results
# Displays the first few rows of the combined results DataFrame
results.head()

# Step 9: Visualize Loss Values
# Plots the loss values against the hyperparameter combinations to visualize performance
results['loss'].plot()                    # Plot the loss values
plt.ylabel('Accuracy')                    # Label the y-axis (consider changing to 'Loss' if applicable)
plt.xlabel('Hyperparam combination')      # Label the x-axis

# Step 10: Retrieve Minimum Loss Value
# Retrieves and displays the minimum loss value from the results stored in the trials object
pd.DataFrame(trials.results)['loss'].min()

**How It All Works Together**
* **Define Hyperparameter Space**: The param_grid specifies a range of values for different hyperparameters that will be evaluated.
* **Objective Function**: The objective function evaluates the model’s performance using a specific combination of hyperparameters.
* **Optimization Process**: The fmin function from Hyperopt minimizes the output of the objective function by exploring the param_grid.
It samples hyperparameters randomly (due to rand.suggest) and evaluates them by calling the objective function.


**Final Model Training**: Once the best hyperparameters are found, they are used to create and train the final model (gbm_final), which is then evaluated for accuracy on the training and test datasets.